[FROM HERE](https://github.com/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb)

In [14]:
from ib_insync import *
util.startLoop()

ib = IB()
# Paper
ib.connect('127.0.0.1', 7496, clientId=12)

# delay data
ib.reqMarketDataType(4)

Suppose we want to find the options on the SPX, with the following conditions:

Use the next three monthly expiries;
Use strike prices within +- 20 dollar of the current SPX value;
Use strike prices that are a multitude of 5 dollar.

To avoid issues with market data permissions, we'll use delayed data:

In [15]:
ib.reqMarketDataType(4)

In [16]:
stock_contract = Stock('TREX', 'SMART', 'USD')
stock_contract

Stock(symbol='TREX', exchange='SMART', currency='USD')

In [17]:
# Qualify the contract
qualified_contracts = ib.qualifyContracts(stock_contract)
stock_contract

Stock(conId=6608603, symbol='TREX', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='TREX', tradingClass='TREX')

qualified contracts - req market data - https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/

In [27]:
if not qualified_contracts:
    print("No qualified contracts found.")
else:
    stock_contract = qualified_contracts[0]

    # Request market data
    data = ib.reqMktData(stock_contract, '', False, False)
    

In [28]:
data.marketPrice()

72.84

chain

In [30]:
chains = ib.reqSecDefOptParams(stock_contract.symbol, '', stock_contract.secType, stock_contract.conId)

display chains

In [31]:
util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,EDGX,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
1,MIAX,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
2,SMART,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
3,BOX,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
4,CBOE2,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
5,SAPPHIRE,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
6,BATS,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
7,IBUSOPT,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
8,MEMX,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."
9,PSE,6608603,TREX,100,"[20250221, 20250321, 20250417, 20250718]","[30.0, 35.0, 40.0, 45.0, 47.5, 50.0, 55.0, 57...."


In [12]:
ib.disconnect()